# Author: Kumar R

# MangoDB Class

In [2]:
# Database (MangoDB) class
class Mangodb:
    
    def __init__(self, db_name = 'SRH_Berlin_University', table_name = 'Parking_Data'):
        self.db_name = db_name
        self.table_name = table_name
    
    # mangoDB database
    def data_upload_to_mangodb(self, record):
        
        try:          
            from pymongo import MongoClient
            client = MongoClient('localhost', 27017)
            db = client[self.db_name]
            collection = db[self.table_name]
            collection.insert_one(record)
            
        except Exception as e:
            print(e)

# Vehicle Class

In [3]:
# vehicle class 
class Vehicle:
    
    def __init__(self, vehicle_type, vehicle_number):
        self.vehicle_type = vehicle_type
        self.vehicle_number = vehicle_number
    
    
    def __repr__(self):
        return f'{self.vehicle_type},{self.vehicle_number}'

# Parking lot class

The parking lot class is responsible only for parking lot related functions but not for any of the vehicle type. It inlcudes the following functions.

    1. Create parking lot table 
    2. upload the data along with check-in time 
    3. display the status 
    4. return the parking lot table and details as a DataFrame whenever we call
    5. price calculation
    6. parking duration calculation
    7. backup the parking lot table for security
    8. check_out

In [4]:
# Parking-lot class 
class Parking:
    
    def __init__(self):
        pass
    
    def create_parking_lot_table(self):   
        
        self.df = pd.DataFrame(index = np.arange(1,376), 
                          columns=['Vehicle_Type', 'Vehicle_Number','Check_In','Check_Out'])

        
    def check_in(self, car_details):

        # Checking if there is any empty space in the parking lot
        if not self.df['Vehicle_Number'].isna().any():
            print("Sorry! The parking lot is full")

        else:
            # Get vehicle_type and number from Vehicle class
            vehicle_type, vehicle_number = str(car_details).split(',')
            
            # Get the last available parking space 
            empty = self.df[self.df.Vehicle_Type.isna()].index
            avilable = empty.max()

            # Allocate the vehicle to the last parking space
            self.df.loc[avilable,'Vehicle_Type'] = vehicle_type
            
            # Assign the vehicle number into parking number
            self.df.loc[avilable, 'Vehicle_Number'] = vehicle_number
            
            # Store the check_in date and time
            self.df.loc[avilable, 'Check_In'] = pd.Timestamp.now()

            print(f'Please park your {vehicle_type} in parking number {avilable}')
            
                   
    def display_parking_lot_capacity(self):
        
        empty_space = self.df.Vehicle_Number.isna().sum()
        filled_space = self.df.Vehicle_Number.notnull().sum() 
        
        return empty_space, filled_space
    
    
    def return_parking_lot_table(self):
        return self.df
    
    @staticmethod
    def parking_price_calculator(duration, vehicle_type):

        # parking price in euro per hour (can be adjustable)
        car = 1
        truck = 1.5
        motor_cycle = 0.7
        
        # Calculate the parking duration in hours
        parking_time = duration.seconds/3600
        
        # parking cost calculation 
        cost = [parking_time*car if vehicle_type== 'car' 
                else parking_time*truck if vehicle_type=='truck' else parking_time*motor_cycle]
        
        return cost
    
    
    
    def parking_duration_calculator(self, parking_lot_number):
        
        # Get the check_in and check_out timings
        park_in = self.df.loc[parking_lot_number[0],'Check_In']
        park_out = self.df.loc[parking_lot_number[0],'Check_Out']
        
        # Calculate the parking duration 
        duration = (park_out-park_in)
        
        return duration
    
    
    def backup(self):

        import os
        time = pd.datetime.now()
        cwd = os.getcwd()
        path = cwd + "/" f'{time.year}.{time.month}.{time.day}.{time.hour}.{time.minute}.{time.second}.{time.microsecond}.csv' #.format(, time.month, time.day, time.hour, time.minute, time.second)
        self.df.to_csv(path)
        print('Took Backup!')
        
        
    def check_out(self, vehicle_number):

        # Search for the vehicle number in the parking_lot
        if self.df.Vehicle_Number.isin([vehicle_number]).any():

            # Get the parking_lot number of the vehicle
            exit_lot = self.df[self.df.Vehicle_Number == vehicle_number].index
            
            # Display parking_lot number
            print(f"Your vehicle is in parking_lot: {exit_lot[0]}")

            print("")
            # Confirm if driver wants to check_out
            exit_type = input('Do you want to exit [y/n]: ')

            if exit_type=='n':
                
                # If the driver doesn't want to check-out
                print("Your vehicle is safe!")
                
            elif exit_type=='y':

                # Record the check_out date and time
                self.df.loc[exit_lot[0], 'Check_Out'] = pd.Timestamp.now()
                
                # Vehicle type to decide the parking price
                vehicle_typ = self.df.loc[exit_lot[0],'Vehicle_Type'] 
                duration = self.parking_duration_calculator(exit_lot)
                
                # Parking price calculation
                parking_cost = self.parking_price_calculator(duration,vehicle_typ) 
                
                # Extract vehicle details as dictionary
                record = self.df.loc[exit_lot[0],:].to_dict()
                record['parking_duration'] = duration
                record['parking_cost_in_euro'] = round(parking_cost[0], 2)

                # Erase the data from parking space as the vehicle check_out
                self.df.loc[exit_lot[0],:]= np.NaN 
                
                print('Thank you! Please visit us again.')
                return record

            else:    
                print("Error! Wrong input")

        else:           
            print()
            print('No vehicle found') # If driver enters wrong number
            print("Please check your vehicle number") # When the vehicle is not parked in the parking_lot

    

In [5]:
def check_in_for_main():
    
    vehicle_type = input("Enter your vehicle type: ")
    vehicle_number = input("Enter your vehicle number: ")
    print()

    car = Vehicle(vehicle_type,vehicle_number)
    garage.check_in(car)

    print('Thank you!')
    
    
def check_out_for_main():
    
    vehicle_num = input("Vehicle number: ")
    bill = garage.check_out(vehicle_num)

    if bill is None:
        pass
    else:
        for i,j in bill.items():
            print(f'{i}:\t\t{j}')

        bill['parking_duration_in_hr'] = bill['parking_duration'].seconds/3600
        bill.pop('parking_duration')
        db = Mangodb() 
        db.data_upload_to_mangodb(bill) 

        
def main():
    
    empty, filled = garage.display_parking_lot_capacity()
    print(f'\tAvailable: \t{empty}\n \tfilled: \t{filled}')  
    
    user_input = input('check_in or check_out ?: ')
    
    if user_input == 'check_in': 
        check_in_for_main()
        
    elif user_input == 'check_out':  
        check_out_for_main()
    
    else:
        print('Error! Wrong input.')


In [6]:
# Run only once to create a parking table
garage = Parking()
garage.create_parking_lot_table()

In [9]:
main()

	Available: 	374
 	filled: 	1
check_in or check_out ?: check_out
Vehicle number: U8374
Your vehicle is in parking_lot: 375

Do you want to exit [y/n]: y
Thank you! Please visit us again.
Vehicle_Type:		car
Vehicle_Number:		U8374
Check_In:		2022-02-05 20:34:42.073682
Check_Out:		2022-02-05 20:34:54.901216
parking_duration:		0 days 00:00:12.827534
parking_cost_in_euro:		0.0


In [8]:
garage.backup()

Took Backup!


<ipython-input-4-8289864390be>:84: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  time = pd.datetime.now()
